In [1]:
import rasterio
import glob
import os,sys
from matplotlib import pyplot as plt
import numpy as np
import fiona
from shapely.geometry import shape
import shapely
from rasterio.mask import mask
from pyproj import Proj, transform

from torch import nn
from torch.nn import functional as F
import torch
from torchvision import models
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import DataLoader

from unet_models import unet11, unet11_MS

from test_unet_helpers import calcXYfromRC, checkWindow, gtDatasetSampler2, DigitalGlobeSamplerTensor, load_pretrained_weights

from utils import variable
from scipy import misc

from gbdxtools import Interface, CatalogImage

from collections import OrderedDict



/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expe

In [2]:
# specify scene id for DG data
dg_scene_id = '1030010057062200' # 2016
dg_scene_id = '104001000AA4B500' # 2015
# dg_scene_id = '103001001A425F00' #napoli 2011

# specify image for 2.0 meter analysis
gt_image_2 = '../rasters/union_impervious_raster_2_0_0_wgs84.tif' # desktop

# specify images for 0.5 meter analysis
gt_image_05 = '../rasters/union_impervious_raster_0_5.tif' # desktop

# specify the shapefile
shpfile = '../union/union.shp' # desktop

with fiona.open(shpfile) as shp:
    crs = shp.crs
    shp_bounds = shp.bounds

In [3]:
# connect to gbdx
gbdx = Interface()

# get the dask array for the 8 band MS image
img_2m = CatalogImage(dg_scene_id, band_type='MS', acomp=True)
rows, cols = img_2m.shape[1:]
chips = img_2m.window_cover((64,64), pad=False)
print(sum(1 for i in chips)) # 96473

#np.mean(img_2m, axis=(1,2)).compute(), np.std(img_2m, axis=(1,2)).compute()

27051


In [6]:
# help(img_2m)

Help on WV03_VNIR in module gbdxtools.images.worldview object:

class WV03_VNIR(WorldViewImage)
 |  A DaskImage is a 2 or 3 dimension dask array that contains implements the `__daskmeta__` interface.
 |  
 |  Method resolution order:
 |      WV03_VNIR
 |      WorldViewImage
 |      gbdxtools.images.base.RDABaseImage
 |      gbdxtools.images.rda_image.RDAImage
 |      gbdxtools.images.meta.GeoDaskImage
 |      gbdxtools.images.meta.DaskImage
 |      dask.array.core.Array
 |      dask.base.DaskMethodsMixin
 |      collections.abc.Container
 |      gbdxtools.images.mixins.geo.PlotMixin
 |      gbdxtools.images.mixins.geo.BandMethodsTemplate
 |      gbdxtools.images.mixins.deprecations.Deprecations
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  __default_options__ = {'acomp': False, 'band_type': 'MS', 'correctionT...
 |  
 |  __driver__ = <gbdxtools.images.drivers.base.WorldViewDriver object>
 |  
 |  __geo__ = <

In [7]:
# # specify image transform for RGB image
# img_transform = Compose([
#     Normalize(mean=[1044.9639, 1271.8239, 1474.1279], std=[824.6721 , 861.0806 , 1008.7456])
# ])

# specify image transform for RGB-NIR
img_transform = Compose([
    Normalize(mean=[1044.9639, 1271.8239, 1474.1279, 2600.4775], std=[824.6721 , 861.0806 , 1008.7456, 1287.1179])
])

# img_transform = Compose([
#     Normalize(mean=[882.8035, 1044.9639, 1271.8239, 1418.305 , 1474.1279, 2069.7122,
#         2600.4775, 2723.9067], std=[711.3716,  824.6721,  861.0806,  961.8498, 1008.7456,  963.6459,
#         1287.1179, 1366.1136])
# ])

# for 2015
# img_transform = Compose([
#     Normalize(mean=[1281.2312, 1243.6267, 1334.2975], std=[645.607, 703.0832, 862.68665])
# ])
img_transform = Compose([
    Normalize(mean=[img_2m.display_stats['mean'][2], img_2m.display_stats['mean'][3],img_2m.display_stats['mean'][5], img_2m.display_stats['mean'][7]], 
                   std=[img_2m.display_stats['sigma'][2], img_2m.display_stats['sigma'][3],img_2m.display_stats['sigma'][5], img_2m.display_stats['sigma'][7]])
])

In [20]:
model_paths = ['runs/debug/for_paper/MS_bgrn_model_e18_b8_no_aug_GPUPAR.pt',
               'runs/debug/for_paper/MS_bgrn_model_e52_b32_no_aug_GPUPAR.pt']

model = unet11_MS(num_bands=4, pretrained=False) # all others
state_dict = torch.load(str(model_path))
new_state_dict = OrderedDict()
for k, v in state_dict['model'].items():
    name = k[7:] # remove 'module.' of dataparallel
    new_state_dict[name]=v

model.load_state_dict(new_state_dict)


num_bands is  4


False

# Load the model and the model state from the .pt file. It may need to be downloaded from AWS S3

In [8]:
## this cell uses GBDX window_cover method

## write out with gdal
import gdal, osr

def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

# RGB-NIR
model_paths = ['runs/debug/for_paper/MS_bgrn_model_e18_b8_no_aug_GPUPAR.pt',
               'runs/debug/for_paper/MS_bgrn_model_e52_b32_no_aug_GPUPAR.pt']


chips = img_2m.window_cover((64,64), pad=False)
    
# 2-19-2019 using batch 32 RGB-NIR model
for model_path in [model_paths[1]]:
    
    torch.cuda.empty_cache()
    
    #model = unet11(pretrained=False) # B-G-R
    model = unet11_MS(num_bands=4, pretrained=False) # all others
    #model = unet11_MS(num_bands=8, pretrained=False) # all others
    
    # load model without gpu weights
    #load_pretrained_weights(model, model_path)
    
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get device for gpu or cpu
#     if torch.cuda.device_count() > 1:
#         print("Let's use", torch.cuda.device_count(), "GPUs!")
#         model = nn.DataParallel(model)
#         model.to(device)

#     elif torch.cuda.is_available():
#         model.to(device)

    model.eval()


#     #load model
#     if os.path.exists(model_path):
#         state = torch.load(str(model_path))
#         epoch = state['epoch']
#         step = state['step']
#         model.load_state_dict(state['model'])
#         print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        
    # load on CPU
    if os.path.exists(model_path):
        state_dict = torch.load(str(model_path))
        epoch = state_dict['epoch']
        step = state_dict['step']
        new_state_dict = OrderedDict()
        for k, v in state_dict['model'].items():
            name = k[7:] # remove 'module.' of dataparallel
            new_state_dict[name]=v

        model.load_state_dict(new_state_dict)
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))

    
    # iterate over the image windows, getting the 4 bands
    for i,chip in enumerate(chips):
        
        rname = '../tiles_64x64/denver_2015_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
        if os.path.exists(rname):
            continue
        
        print('on {} of {}'.format(i+1, 27051))
        img_arr = chip[[1,2,4,6], :, :].compute() #B-G-R-N1
        img_arr = img_transform(torch.from_numpy(img_arr))
        
        #input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0)
        input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0).cpu()
        del img_arr

        # model inference
        big_mask = model(input_img)

        # file creation
        raster_id = os.path.basename(model_path).split('.')[0]
        rname = '../tiles_64x64/denver_2016_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
        aff = chip.affine
        raster_origin = (aff.c, aff.f)
        pixel_height = aff.e
        pixel_width = aff.a
        array2raster(rname, raster_origin, pixel_width, pixel_height, big_mask.cpu().detach().numpy().squeeze())

        del input_img
        del big_mask
        

        torch.cuda.empty_cache()

num_bands is  4
Restored model, epoch 51, step 13,000
on 1 of 27051


/home/ubuntu/segmentation/IS_segmentation/utils.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return cuda(Variable(x, volatile=volatile))


on 2 of 27051
on 3 of 27051
on 4 of 27051
on 5 of 27051
on 6 of 27051
on 7 of 27051
on 8 of 27051
on 9 of 27051
on 10 of 27051
on 11 of 27051
on 12 of 27051
on 13 of 27051
on 14 of 27051
on 15 of 27051
on 16 of 27051
on 17 of 27051
on 18 of 27051
on 19 of 27051
on 20 of 27051
on 21 of 27051
on 22 of 27051
on 23 of 27051
on 24 of 27051
on 25 of 27051
on 26 of 27051
on 27 of 27051
on 28 of 27051
on 29 of 27051
on 30 of 27051
on 31 of 27051
on 32 of 27051
on 33 of 27051
on 34 of 27051
on 35 of 27051
on 36 of 27051
on 37 of 27051
on 38 of 27051
on 39 of 27051
on 40 of 27051
on 41 of 27051
on 42 of 27051
on 43 of 27051
on 44 of 27051
on 45 of 27051
on 46 of 27051
on 47 of 27051
on 48 of 27051
on 49 of 27051
on 50 of 27051
on 51 of 27051
on 52 of 27051
on 53 of 27051
on 54 of 27051
on 55 of 27051
on 56 of 27051
on 57 of 27051
on 58 of 27051
on 59 of 27051
on 60 of 27051
on 61 of 27051
on 62 of 27051
on 63 of 27051
on 64 of 27051
on 65 of 27051
on 66 of 27051
on 67 of 27051
on 68 of 27051
on 

on 521 of 27051
on 522 of 27051
on 523 of 27051
on 524 of 27051
on 525 of 27051
on 526 of 27051
on 527 of 27051
on 528 of 27051
on 529 of 27051
on 530 of 27051
on 531 of 27051
on 532 of 27051
on 533 of 27051
on 534 of 27051
on 535 of 27051
on 536 of 27051
on 537 of 27051
on 538 of 27051
on 539 of 27051
on 540 of 27051
on 541 of 27051
on 542 of 27051
on 543 of 27051
on 544 of 27051
on 545 of 27051
on 546 of 27051
on 547 of 27051
on 548 of 27051
on 549 of 27051
on 550 of 27051
on 551 of 27051
on 552 of 27051
on 553 of 27051
on 554 of 27051
on 555 of 27051
on 556 of 27051
on 557 of 27051
on 558 of 27051
on 559 of 27051
on 560 of 27051
on 561 of 27051
on 562 of 27051
on 563 of 27051
on 564 of 27051
on 565 of 27051
on 566 of 27051
on 567 of 27051
on 568 of 27051
on 569 of 27051
on 570 of 27051
on 571 of 27051
on 572 of 27051
on 573 of 27051
on 574 of 27051
on 575 of 27051
on 576 of 27051
on 577 of 27051
on 578 of 27051
on 579 of 27051
on 580 of 27051
on 581 of 27051
on 582 of 27051
on 583 o

on 1033 of 27051
on 1034 of 27051
on 1035 of 27051
on 1036 of 27051
on 1037 of 27051
on 1038 of 27051
on 1039 of 27051
on 1040 of 27051
on 1041 of 27051
on 1042 of 27051
on 1043 of 27051
on 1044 of 27051
on 1045 of 27051
on 1046 of 27051
on 1047 of 27051
on 1048 of 27051
on 1049 of 27051
on 1050 of 27051
on 1051 of 27051
on 1052 of 27051
on 1053 of 27051
on 1054 of 27051
on 1055 of 27051
on 1056 of 27051
on 1057 of 27051
on 1058 of 27051
on 1059 of 27051
on 1060 of 27051
on 1061 of 27051
on 1062 of 27051
on 1063 of 27051
on 1064 of 27051
on 1065 of 27051
on 1066 of 27051
on 1067 of 27051
on 1068 of 27051
on 1069 of 27051
on 1070 of 27051
on 1071 of 27051
on 1072 of 27051
on 1073 of 27051
on 1074 of 27051
on 1075 of 27051
on 1076 of 27051
on 1077 of 27051
on 1078 of 27051
on 1079 of 27051
on 1080 of 27051
on 1081 of 27051
on 1082 of 27051
on 1083 of 27051
on 1084 of 27051
on 1085 of 27051
on 1086 of 27051
on 1087 of 27051
on 1088 of 27051
on 1089 of 27051
on 1090 of 27051
on 1091 of 270

...

on 24813 of 27051
on 24814 of 27051
on 24815 of 27051
on 24816 of 27051
on 24817 of 27051
on 24818 of 27051
on 24819 of 27051
on 24820 of 27051
on 24821 of 27051
on 24822 of 27051
on 24823 of 27051
on 24824 of 27051
on 24825 of 27051
on 24826 of 27051
on 24827 of 27051
on 24828 of 27051
on 24829 of 27051
on 24830 of 27051
on 24831 of 27051
on 24832 of 27051
on 24833 of 27051
on 24834 of 27051
on 24835 of 27051
on 24836 of 27051
on 24837 of 27051
on 24838 of 27051
on 24839 of 27051
on 24840 of 27051
on 24841 of 27051
on 24842 of 27051
on 24843 of 27051
on 24844 of 27051
on 24845 of 27051
on 24846 of 27051
on 24847 of 27051
on 24848 of 27051
on 24849 of 27051
on 24850 of 27051
on 24851 of 27051
on 24852 of 27051
on 24853 of 27051
on 24854 of 27051
on 24855 of 27051
on 24856 of 27051
on 24857 of 27051
on 24858 of 27051
on 24859 of 27051
on 24860 of 27051
on 24861 of 27051
on 24862 of 27051
on 24863 of 27051
on 24864 of 27051
on 24865 of 27051
on 24866 of 27051
on 24867 of 27051
on 24868 o

on 25270 of 27051
on 25271 of 27051
on 25272 of 27051
on 25273 of 27051
on 25274 of 27051
on 25275 of 27051
on 25276 of 27051
on 25277 of 27051
on 25278 of 27051
on 25279 of 27051
on 25280 of 27051
on 25281 of 27051
on 25282 of 27051
on 25283 of 27051
on 25284 of 27051
on 25285 of 27051
on 25286 of 27051
on 25287 of 27051
on 25288 of 27051
on 25289 of 27051
on 25290 of 27051
on 25291 of 27051
on 25292 of 27051
on 25293 of 27051
on 25294 of 27051
on 25295 of 27051
on 25296 of 27051
on 25297 of 27051
on 25298 of 27051
on 25299 of 27051
on 25300 of 27051
on 25301 of 27051
on 25302 of 27051
on 25303 of 27051
on 25304 of 27051
on 25305 of 27051
on 25306 of 27051
on 25307 of 27051
on 25308 of 27051
on 25309 of 27051
on 25310 of 27051
on 25311 of 27051
on 25312 of 27051
on 25313 of 27051
on 25314 of 27051
on 25315 of 27051
on 25316 of 27051
on 25317 of 27051
on 25318 of 27051
on 25319 of 27051
on 25320 of 27051
on 25321 of 27051
on 25322 of 27051
on 25323 of 27051
on 25324 of 27051
on 25325 o

on 25726 of 27051
on 25727 of 27051
on 25728 of 27051
on 25729 of 27051
on 25730 of 27051
on 25731 of 27051
on 25732 of 27051
on 25733 of 27051
on 25734 of 27051
on 25735 of 27051
on 25736 of 27051
on 25737 of 27051
on 25738 of 27051
on 25739 of 27051
on 25740 of 27051
on 25741 of 27051
on 25742 of 27051
on 25743 of 27051
on 25744 of 27051
on 25745 of 27051
on 25746 of 27051
on 25747 of 27051
on 25748 of 27051
on 25749 of 27051
on 25750 of 27051
on 25751 of 27051
on 25752 of 27051
on 25753 of 27051
on 25754 of 27051
on 25755 of 27051
on 25756 of 27051
on 25757 of 27051
on 25758 of 27051
on 25759 of 27051
on 25760 of 27051
on 25761 of 27051
on 25762 of 27051
on 25763 of 27051
on 25764 of 27051
on 25765 of 27051
on 25766 of 27051
on 25767 of 27051
on 25768 of 27051
on 25769 of 27051
on 25770 of 27051
on 25771 of 27051
on 25772 of 27051
on 25773 of 27051
on 25774 of 27051
on 25775 of 27051
on 25776 of 27051
on 25777 of 27051
on 25778 of 27051
on 25779 of 27051
on 25780 of 27051
on 25781 o

on 26182 of 27051
on 26183 of 27051
on 26184 of 27051
on 26185 of 27051
on 26186 of 27051
on 26187 of 27051
on 26188 of 27051
on 26189 of 27051
on 26190 of 27051
on 26191 of 27051
on 26192 of 27051
on 26193 of 27051
on 26194 of 27051
on 26195 of 27051
on 26196 of 27051
on 26197 of 27051
on 26198 of 27051
on 26199 of 27051
on 26200 of 27051
on 26201 of 27051
on 26202 of 27051
on 26203 of 27051
on 26204 of 27051
on 26205 of 27051
on 26206 of 27051
on 26207 of 27051
on 26208 of 27051
on 26209 of 27051
on 26210 of 27051
on 26211 of 27051
on 26212 of 27051
on 26213 of 27051
on 26214 of 27051
on 26215 of 27051
on 26216 of 27051
on 26217 of 27051
on 26218 of 27051
on 26219 of 27051
on 26220 of 27051
on 26221 of 27051
on 26222 of 27051
on 26223 of 27051
on 26224 of 27051
on 26225 of 27051
on 26226 of 27051
on 26227 of 27051
on 26228 of 27051
on 26229 of 27051
on 26230 of 27051
on 26231 of 27051
on 26232 of 27051
on 26233 of 27051
on 26234 of 27051
on 26235 of 27051
on 26236 of 27051
on 26237 o

on 26638 of 27051
on 26639 of 27051
on 26640 of 27051
on 26641 of 27051
on 26642 of 27051
on 26643 of 27051
on 26644 of 27051
on 26645 of 27051
on 26646 of 27051
on 26647 of 27051
on 26648 of 27051
on 26649 of 27051
on 26650 of 27051
on 26651 of 27051
on 26652 of 27051
on 26653 of 27051
on 26654 of 27051
on 26655 of 27051
on 26656 of 27051
on 26657 of 27051
on 26658 of 27051
on 26659 of 27051
on 26660 of 27051
on 26661 of 27051
on 26662 of 27051
on 26663 of 27051
on 26664 of 27051
on 26665 of 27051
on 26666 of 27051
on 26667 of 27051
on 26668 of 27051
on 26669 of 27051
on 26670 of 27051
on 26671 of 27051
on 26672 of 27051
on 26673 of 27051
on 26674 of 27051
on 26675 of 27051
on 26676 of 27051
on 26677 of 27051
on 26678 of 27051
on 26679 of 27051
on 26680 of 27051
on 26681 of 27051
on 26682 of 27051
on 26683 of 27051
on 26684 of 27051
on 26685 of 27051
on 26686 of 27051
on 26687 of 27051
on 26688 of 27051
on 26689 of 27051
on 26690 of 27051
on 26691 of 27051
on 26692 of 27051
on 26693 o

In [10]:
input_img.cpu()

tensor([[[[-0.7372, -0.7202, -0.7869,  ..., -0.7603, -0.6305, -0.5371],
          [-0.5359, -0.6487, -0.7700,  ..., -0.8257, -0.7809, -0.8536],
          [-0.3395, -0.6414, -0.8500,  ..., -0.3031, -0.0909, -0.0206],
          ...,
          [-0.8900, -0.9143, -0.9324,  ..., -0.4620, -0.3371, -0.2922],
          [-0.9773, -0.9458, -0.8949,  ..., -0.5008, -0.2485, -0.1515],
          [-0.8379, -0.8985, -0.9349,  ..., -0.4668, -0.3771, -0.4983]],

         [[-0.1670,  0.0525, -0.1275,  ..., -0.6908, -0.5212, -0.4272],
          [ 0.2046,  0.2627, -0.0869,  ..., -0.7988, -0.6594, -0.7175],
          [ 0.0316, -0.0753, -0.2541,  ..., -0.1299,  0.1500,  0.2998],
          ...,
          [-0.8441, -0.9660, -1.0299,  ..., -0.3737, -0.3459, -0.4272],
          [-0.8952, -0.8406, -0.8081,  ..., -0.4504, -0.2994, -0.3877],
          [-0.9300, -0.8522, -0.8174,  ..., -0.4945, -0.4713, -0.6571]],

         [[-0.6247, -0.6227, -0.6822,  ..., -0.8209, -0.5325, -0.3124],
          [-0.1627, -0.4442, -

In [7]:
## try this in a loop
# model_path = 'runs/debug/model_e100_b32_no_aug_GPUPAR.pt' # RGB batch size 32
# model_path = 'runs/debug/model_e100_b8_no_aug_GPUPAR.pt' # RGB batch size 8
# model_path = 'runs/debug/MS_model_e100_b8_no_aug_GPUPAR.pt' # MS model
# model_path = 'runs/debug/model_e44_s34938.pt' # RGB early exit (large batch size, i think)

# #for paper, batch sizes
# model_paths = ['runs/debug/for_paper/BGR_model_e20_b8_no_aug_GPUPAR.pt',
#                'runs/debug/for_paper/MS_bgrn_model_e18_b8_no_aug_GPUPAR.pt',
#                'runs/debug/for_paper/MS2_model_e100_b8_no_aug_GPUPAR.pt',
#                'runs/debug/for_paper/BGR_model_e64_b32_no_aug_GPUPAR.pt',
#                'runs/debug/for_paper/MS_bgrn_model_e52_b32_no_aug_GPUPAR.pt',
#                'runs/debug/for_paper/MS2_model_e100_b32_no_aug_GPUPAR.pt']

# RGB only
model_paths = ['runs/debug/for_paper/BGR_model_e20_b8_no_aug_GPUPAR.pt',
               'runs/debug/for_paper/BGR_model_e64_b32_no_aug_GPUPAR.pt']

# RGB-NIR
model_paths = ['runs/debug/for_paper/MS_bgrn_model_e18_b8_no_aug_GPUPAR.pt',
               'runs/debug/for_paper/MS_bgrn_model_e52_b32_no_aug_GPUPAR.pt']

# all 8
# model_paths = ['runs/debug/for_paper/MS2_model_e100_b8_no_aug_GPUPAR.pt',
#                'runs/debug/for_paper/MS2_model_e100_b32_no_aug_GPUPAR.pt']

# 2-19-2019 using batch 32 RGB-NIR model
for model_path in [model_paths[1]]:
    
    torch.cuda.empty_cache()
    
    #model = unet11(pretrained=False) # B-G-R
    model = unet11_MS(num_bands=4, pretrained=False) # all others
    #model = unet11_MS(num_bands=8, pretrained=False) # all others
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get device for gpu or cpu
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
        model.to(device)

    elif torch.cuda.is_available():
        model.to(device)

    model.eval()


    #load model
    if os.path.exists(model_path):
        state = torch.load(str(model_path))
        epoch = state['epoch']
        step = state['step']
        model.load_state_dict(state['model'])
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))
        
    #img_arr = img_2m[[1,2,4], :2048,:2048].compute() #B-G-R
    #img_arr = img_2m[[1,2,4,6], :2048,:2048].compute() #B-G-R-N1
    img_arr = img_2m[:, :2048,:2048].compute() # all 8
    img_arr = img_transform(torch.from_numpy(img_arr))

    input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0)
    del img_arr
    
    big_mask = model(input_img)
    
    big_mask.cpu().detach().numpy().squeeze().shape
    
    ## write out with gdal
    import gdal, osr

    def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

        cols = array.shape[1]
        rows = array.shape[0]
        originX = rasterOrigin[0]
        originY = rasterOrigin[1]

        driver = gdal.GetDriverByName('GTiff')
        outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
        outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
        outband = outRaster.GetRasterBand(1)
        outband.WriteArray(array)
        outRasterSRS = osr.SpatialReference()
        outRasterSRS.ImportFromEPSG(4326)
        outRaster.SetProjection(outRasterSRS.ExportToWkt())
        outband.FlushCache()

    raster_id = os.path.basename(model_path).split('.')[0]
    rname = 'runs/debug/for_paper/denver_2016_{}.tif'.format(raster_id)
    aff = img_2m.affine
    raster_origin = (aff.c, aff.f)
    pixel_height = aff.e
    pixel_width = aff.a
    array2raster(rname, raster_origin, pixel_width, pixel_height, big_mask.cpu().detach().numpy().squeeze())
    
    del input_img
    del big_mask
    del model
    
    torch.cuda.empty_cache()

num_bands is  8
Let's use 4 GPUs!
Restored model, epoch 99, step 100,000


/home/ubuntu/segmentation/IS_segmentation/utils.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return cuda(Variable(x, volatile=volatile))


num_bands is  8
Let's use 4 GPUs!
Restored model, epoch 99, step 25,250


In [8]:
del input_img
del big_mask

In [7]:
model_path = 'runs/debug/model_e100_b32_no_aug_GPUPAR.pt' # RGB batch size 32
model_path = 'runs/debug/model_e100_b8_no_aug_GPUPAR.pt' # RGB batch size 8
model_path = 'runs/debug/MS_model_e100_b8_no_aug_GPUPAR.pt' # MS model
model_path = 'runs/debug/model_e44_s34938.pt' # RGB early exit (large batch size, i think)



model = unet11(pretrained=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get device for gpu or cpu
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    model.to(device)
    
elif torch.cuda.is_available():
    model.to(device)

model.eval()


#load model
if os.path.exists(model_path):
    state = torch.load(str(model_path))
    epoch = state['epoch']
    step = state['step']
    model.load_state_dict(state['model'])
    print('Restored model, epoch {}, step {:,}'.format(epoch, step))

Restored model, epoch 44, step 537


In [8]:
img_2m.shape, torch.cuda.is_available()

((8, 3333, 4579), True)

In [9]:
## try to the full dataset
# input_img = torch.unsqueeze(variable(dg_dataset[test_ind], volatile=True), dim=0)



img_arr = img_2m[[1,2,4], :2048,:2048].compute() #BGR
# img_arr = img_2m[:, :2048,:2048].compute() # all 8
img_arr = img_transform(torch.from_numpy(img_arr))


In [10]:
input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0)
del img_arr

/home/ubuntu/segmentation/IS_segmentation/utils.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return cuda(Variable(x, volatile=volatile))


In [11]:
# model.cpu()
# big_mask = model(input_img.cpu())

big_mask = model(input_img)

In [12]:
big_mask.cpu().detach().numpy().squeeze().shape

(2048, 2048)

In [13]:
## write out with gdal
import gdal, osr

def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
rname = 'runs/debug/test_eval_BGR_2015_renorm.tif'
aff = img_2m.affine
raster_origin = (aff.c, aff.f)
pixel_height = aff.e
pixel_width = aff.a
array2raster(rname, raster_origin, pixel_width, pixel_height, big_mask.cpu().detach().numpy().squeeze())